## CompEL Simulations Analysis

In [1]:
import json
import glob
import numpy as np
import pandas as pd
import MDAnalysis as mda
from MDAnalysis.analysis import distances
# import nglview as nv
# import lipyphilic as lpp
import matplotlib.pyplot as plt
from scipy.spatial import distance # to calculate pore
from MDAnalysis.analysis import density
import subprocess # To run amber & gromacs commands from jupyter notebook
import MDAnalysis.lib.util as util # To convert from 3 letter to 1 letter aa code
from MDAnalysis.analysis.hydrogenbonds.hbond_analysis import HydrogenBondAnalysis as HBA
import matplotlib.patheffects as pe # Cooler plots
# For lipid occupancy
from pylipid.api import LipidInteraction
# For RMSD
import MDAnalysis.analysis.rms
# For SS
import matplotlib.colors as mcolors
import pickle # To save dictionary into pickle file
from matplotlib.lines import Line2D # For SS average plot
# To analyse membrane thickness and area per lipid
from lipyphilic.lib.assign_leaflets import AssignLeaflets
from lipyphilic.lib.area_per_lipid import AreaPerLipid
from lipyphilic.lib.memb_thickness import MembThickness

In [2]:
folders = glob.glob('1.POPC/2.CompEL/[0-9]*.*') # Find folders that start with a number and a dot
folders.sort(reverse=False, key=lambda x: int(x.split('/')[-1].split('.')[0]))
print(f'Analysis can be run in folders: {folders}')
peptide_names = ['Control','Arg9','MAP','TP10','TP2','Leu9']
all_residue_labels = [[], ['1R','2R','3R','4R','5R','6R','7R','8R','9R'],\
                          ['1K','2L','3A','4L','5K','6L','7A','8L','9K','10A','11L','12K','13A','14A','15L','16K','17L','18A'],\
                          ['1A','2G','3Y','4L','5L','6G','7K','8I','9N','10L','11K','12A','13L','14A','15A','16L','17A','18K','19K','20I','21L'],\
                          ['1P','2L','3I','4Y','5L','6R','7L','8L','9R','10G','11Q','12W','13C'],\
                          ['1L','2L','3L','4L','5L','6L','7L','8L','9L'],]
cpp_names = peptide_names[1:]
color_names = ["Pink","Blue", "Green", "Orange", "purple", "red"]
pept_colors = ["pink","royalblue", "lime", "orange", "purple", "red"]
peptides_length = [0,9,18,21,13,9]
data_folder = '4.Analysis_data/'
plots_folder = '5.Analysis_plots/'

Analysis can be run in folders: ['1.POPC/2.CompEL/0.0DQ', '1.POPC/2.CompEL/1.4DQ', '1.POPC/2.CompEL/2.8DQ', '1.POPC/2.CompEL/3.12DQ', '1.POPC/2.CompEL/4.16DQ', '1.POPC/2.CompEL/5.20DQ', '1.POPC/2.CompEL/6.24DQ', '1.POPC/2.CompEL/7.16DQSeveralPeptides', '1.POPC/2.CompEL/8.12DQSeveralPeptides', '1.POPC/2.CompEL/9.8DQSeveralPeptides', '1.POPC/2.CompEL/10.0DQSeveralPeptides']


In [3]:
popc_atoms = [
    'N', 'C12', 'H12A', 'H12B', 'C13', 'H13A', 'H13B', 'H13C', 'C14', 'H14A', 'H14B', 'H14C', 
    'C15', 'H15A', 'H15B', 'H15C', 'C11', 'H11A', 'H11B', 'P', 'O13', 'O14', 'O12', 'O11', 'C1', 
    'HA', 'HB', 'C2', 'HS', 'O21', 'C21', 'O22', 'C22', 'H2R', 'H2S', 'C3', 'HX', 'HY', 'O31', 
    'C31', 'O32', 'C32', 'H2X', 'H2Y', 'C23', 'H3R', 'H3S', 'C24', 'H4R', 'H4S', 'C25', 'H5R', 
    'H5S', 'C26', 'H6R', 'H6S', 'C27', 'H7R', 'H7S', 'C28', 'H8R', 'H8S', 'C29', 'H91', 'C210', 
    'H101', 'C211', 'H11R', 'H11S', 'C212', 'H12R', 'H12S', 'C213', 'H13R', 'H13S', 'C214', 
    'H14R', 'H14S', 'C215', 'H15R', 'H15S', 'C216', 'H16R', 'H16S', 'C217', 'H17R', 'H17S', 
    'C218', 'H18R', 'H18S', 'H18T', 'C33', 'H3X', 'H3Y', 'C34', 'H4X', 'H4Y', 'C35', 'H5X', 
    'H5Y', 'C36', 'H6X', 'H6Y', 'C37', 'H7X', 'H7Y', 'C38', 'H8X', 'H8Y', 'C39', 'H9X', 'H9Y', 
    'C310', 'H10X', 'H10Y', 'C311', 'H11X', 'H11Y', 'C312', 'H12X', 'H12Y', 'C313', 'H13X', 
    'H13Y', 'C314', 'H14X', 'H14Y', 'C315', 'H15X', 'H15Y', 'C316', 'H16X', 'H16Y', 'H16Z'
]
head_atoms = popc_atoms[:42]
tail_atoms = popc_atoms[42:]

In [8]:
# dq_num = 4
# pept_num = 1
# repl_num = 1

# folder = f'{folders[dq_num]}/'
# pept_folders = sorted(glob.glob(f'{folder}/?.*'))
# pept_folder = f'{pept_folders[pept_num]}/'
# print(pept_folder + " Replica " + str(repl_num))

# # parm file
# if dq_num < 7: # If DQ lower than 7 the name is:
#     if pept_num == 0: parm_file = f'{pept_folder}system-kcl.gro' # If no peptide, then system-kcl.gro
#     elif pept_num == 5: # If Leu9, then system-peptide.gro, since no Cl- added
#         if repl_num == 2 and dq_num == 0: # Special case
#             parm_file = f'{pept_folder}step6.6_equilibration_2.gro'
#         else:
#             parm_file = f'{pept_folder}system-peptide.gro'
#     else: parm_file = f'{pept_folder}system-clpeptide.gro' # If peptide, then system-clpeptide.gro, since Cl- added
# else: # Different parameter files for DQ bigger than 6
#     if pept_num == 5: # Leu9 has no Cl added, so different files
#         parm_file = f'{pept_folder}8peptides.gro'
#     else: parm_file = f'{pept_folder}8peptides-cl.gro'
# # traj file
# if repl_num == 1:
#     if pept_num == 0:
#         if dq_num == 2 or dq_num == 4:
#             traj_file = [f'{pept_folder}step7_compel_first.xtc',f'{pept_folder}step7_compel_second.xtc']
#         else:
#             traj_file = f'{pept_folder}step7_compel.xtc'
#     elif pept_num == 5 and dq_num == 6: 
#         traj_file = [f'{pept_folder}step7_compel_first.xtc',f'{pept_folder}step7_compel_second.xtc']
#     else: traj_file = f'{pept_folder}step7_compel.xtc'
# elif repl_num == 2: 
#     if pept_num == 0 or pept_num == 5:
#         traj_file = [f'{pept_folder}step7_compel_2_first.xtc',f'{pept_folder}step7_compel_2_second.xtc']
#     else:
#         traj_file = f'{pept_folder}step7_compel_2.xtc'
# else: 
#     if dq_num == 4: 
#         traj_file = f'{pept_folder}step7_compel_3.xtc'
#     else: print(f'Does this dq_num ({dq_num}) have a third replica?')
# if repl_num == 1: center_traj = f'{pept_folder}step7_compel-whole-nojump-mol.xtc'
# else: center_traj = f'{pept_folder}step7_compel-whole-nojump-mol_{repl_num}.xtc'
# # print(traj_file,center_traj)

1.POPC/2.CompEL/4.16DQ/1.Arg9/ Replica 1


In [10]:
# u = mda.Universe(parm_file)
# u = mda.Universe(parm_file,traj_file)
# c = mda.Universe(parm_file,center_traj)

# peptide = c.select_atoms('protein')
# upper_membrane = c.select_atoms('resname POPC')[:34304] # There are 34304 atoms in each membrane
# lower_membrane = c.select_atoms('resname POPC')[34304:]

## Prepare System

In [8]:
def prepare_system(dq_num,pept_num,part):
    # Load universe
    folder = f'{folders[dq_num]}/'
    pept_folders = sorted(glob.glob(f'{folder}/?.*'))
    pept_folder = f'{pept_folders[pept_num]}/'
    print(pept_folder)
    if dq_num == 7 or dq_num == 8 or dq_num == 9 or dq_num == 10: # For several peptides
        if pept_num == 5: parm_file = f'{pept_folder}8peptides.gro'
        elif pept_num == 0: pass
        else: parm_file = f'{pept_folder}8peptides-cl.gro'
    elif part == 0:
        if pept_num == 0:
            parm_file = f'{pept_folder}system-kcl.gro'
        elif pept_num == 5: parm_file = f'{pept_folder}system-peptide.gro'
        else: parm_file = f'{pept_folder}system-clpeptide.gro'
    
        
    else: parm_file = f'{pept_folder}step6.6_equilibration.gro'
    print(parm_file)
    traj_file = f'{pept_folder}step7_compel.xtc'
    u = mda.Universe(parm_file)

    # Select upper and lower membrane
    memb1 = u.select_atoms('resname POPC').residues[:256]
    memb2 = u.select_atoms('resname POPC').residues[256:]
    memb1_z = memb1.center_of_mass()[2]
    memb2_z = memb2.center_of_mass()[2]
    # Cl ions
    cl_ions = u.select_atoms('resname CLA')
    cl_ions_1 = 0
    cl_ions_2 = 0
    for resid in cl_ions.resids:
        z_cl = u.select_atoms(f'resid {resid}').center_of_mass()[2]
        if z_cl > memb1_z and z_cl < memb2_z: # If it's in the middle water compartment
            cl_ions_1 +=1
        else: cl_ions_2 += 1
    # K ions
    k_ions = u.select_atoms('resname POT')
    k_ions_1 = 0
    k_ions_2 = 0
    for resid in k_ions.resids:
        z_k = u.select_atoms(f'resid {resid}').center_of_mass()[2]
        if z_k > memb1_z and z_k < memb2_z:
            k_ions_1 +=1
        else: k_ions_2 += 1
    # Calc delta q
    delta_q = k_ions_1 - cl_ions_1 + (cl_ions_2 - k_ions_2)
    # Get resids of membrane lipids
    memb1_resids = [i for i in u.select_atoms('resname POPC').residues.resids if i < 5000]
    memb2_resids = [i for i in u.select_atoms('resname POPC').residues.resids if i > 5000]
    # print(f'r {memb1_resids[0]}-{memb1_resids[-1]}')
    print(f'r {memb2_resids[0]}-{memb2_resids[-1]}')
    # Get number of waters in each compartment. If resid is lower than resid of first POPC in memb2, then this water is in compartment 1.
    water1_resids = [i for i in u.select_atoms('resname TIP3').residues.resids if i < memb2_resids[0]]
    water2_resids = [i for i in u.select_atoms('resname TIP3').residues.resids if i > memb2_resids[0]]
    print(len(water1_resids),len(water2_resids))

    # Also, generate system.info file
    box_dimensions = list(u.dimensions)
    total_number_atoms = len(u.select_atoms('all').atoms)
    total_number_residues = len(u.select_atoms('all').residues)
    total_waters = len(u.select_atoms('resname TIP3').residues)
    prot_atoms = len(u.select_atoms('protein'))
    k_atoms = len(u.select_atoms('resname POT'))
    cl_atoms = len(u.select_atoms('resname CLA'))
    lipid_atoms = len(u.select_atoms('resname POPC'))
    lipid_residues = len(u.select_atoms('resname POPC').residues)
    data = {
    "box_dimensions": f'{box_dimensions[:3]} Å',
    # "len_traj": len_traj,
    # "time_traj": f'{sim_time} ns',
    "total_number_atoms": total_number_atoms,
    "total_number_residues": total_number_residues,
    "total_waters": total_waters,
    "cl_atoms": cl_atoms,
    "k_atoms": k_atoms,
    "cl_ions_1": cl_ions_1,
    "cl_ions_2": cl_ions_2,
    "k_ions_1": k_ions_1,
    "k_ions_2": k_ions_2,
    "delta_q": delta_q,
    "prot_atoms": prot_atoms,
    "lipid_atoms": lipid_atoms,
    "lipid_residues": lipid_residues,

    }

    with open(f"{data_folder}0.SysInfo/{dq_num}.{pept_num}.{part}_data.json", "w") as json_file:
        json.dump(data, json_file, indent=4)
        

In [9]:
# This is to generate the systems with the correct number of ions
# for i in range(7):
#     print(f'gmx insert-molecules -f system.gro -o {i}.{i*4}DQ/system-cl.gro -ci cl.gro -ip {i}.{i*4}DQ/positions-cl.dat -replace TIP3 -dr 2 2 1')
#     print(f'gmx insert-molecules -f {i}.{i*4}DQ/system-cl.gro -o {i}.{i*4}DQ/system-kcl.gro -ci k.gro -ip {i}.{i*4}DQ/positions-k.dat -replace TIP3 -dr 2 2 1')

In [10]:
# prepare_system(5,5,0)

## Trajectory wrapping & centering

In [11]:
def center_wrap_trajectory(pept_folder,repl_num):
    if repl_num == 1:
        # First, make molecules whole (so that they don't break) and center the membranes in the system
        gmx_command_1 = f'(echo "POPC"; echo "System") | gmx trjconv -f {pept_folder}step7_compel.xtc -s {pept_folder}step7_compel.tpr -o {pept_folder}step7_compel-whole.xtc -pbc whole -n {pept_folder}index.ndx -center'
        # Second, make that molecules do not jump from one PBC to the other
        gmx_command_2 = f'(echo "System") | gmx trjconv -f {pept_folder}step7_compel-whole.xtc -s {pept_folder}step7_compel.tpr -o {pept_folder}step7_compel-whole-nojump.xtc -pbc nojump'
        # Third, put all molecules in the same box
        gmx_command_3 = f'(echo "System") | gmx trjconv -f {pept_folder}step7_compel-whole-nojump.xtc -s {pept_folder}step7_compel.tpr -o {pept_folder}step7_compel-whole-nojump-mol.xtc -pbc mol'
    else:
        gmx_command_1 = f'(echo "POPC"; echo "System") | gmx trjconv -f {pept_folder}step7_compel_{repl_num}.xtc -s {pept_folder}step7_compel_{repl_num}.tpr -o {pept_folder}step7_compel-whole_{repl_num}.xtc -pbc whole -n {pept_folder}index.ndx -center'
        gmx_command_2 = f'(echo "System") | gmx trjconv -f {pept_folder}step7_compel-whole_{repl_num}.xtc -s {pept_folder}step7_compel_{repl_num}.tpr -o {pept_folder}step7_compel-whole-nojump_{repl_num}.xtc -pbc nojump'
        gmx_command_3 = f'(echo "System") | gmx trjconv -f {pept_folder}step7_compel-whole-nojump_{repl_num}.xtc -s {pept_folder}step7_compel_{repl_num}.tpr -o {pept_folder}step7_compel-whole-nojump-mol_{repl_num}.xtc -pbc mol'
    # just another option for first command if we need to skip some frames or read until certain frame
    # gmx_command_1 = 'gmx trjconv -f step7_compel.xtc -s step7_compel_first.tpr -o step7_compel-whole.xtc -pbc whole -e 250000 -skip 10 -n index.ndx -center '
    
    subprocess.call(f'/bin/bash -c "source /usr/local/gromacs-2020.7/bin/GMXRC && {gmx_command_1}"', shell=True)
    subprocess.call(f'/bin/bash -c "source /usr/local/gromacs-2020.7/bin/GMXRC && {gmx_command_2}"', shell=True)
    subprocess.call(f'/bin/bash -c "source /usr/local/gromacs-2020.7/bin/GMXRC && {gmx_command_3}"', shell=True)

## RMSD

In [13]:
def analyse_rmsd(u,dq_num,pept_num,repl_num):
    """ Analyse the RMSD of every simulation """
    # Run analysis
    R = MDAnalysis.analysis.rms.RMSD(u,select='all',groupselections=['all','resname POPC','protein'])
    R.run()
    rmsd = R.rmsd.T # transpose makes it easier for plotting
    time = [i for i in range(len(u.trajectory))]

    # Save to df
    df = pd.DataFrame()
    df['Time'] = time
    df['All'] = rmsd[3]
    df['Membrane'] = rmsd[4]
    if pept_num != 0:
        df['Peptide'] = rmsd[5]
    df.to_csv(f'{data_folder}1.RMSD/{dq_num}.{pept_num}.{repl_num}.csv')

In [14]:
def plot_rmsd(dq_num,pept_num,repl_num,sliding_window=10):
    df = pd.read_csv(f'{data_folder}1.RMSD/{dq_num}.{pept_num}.{repl_num}.csv')
    df_rolling = df.rolling(sliding_window).mean()
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(df_rolling['Time'], df_rolling['All'],label='All',color='b',path_effects=[pe.Stroke(linewidth=3, foreground='black'), pe.Normal()])
    ax.plot(df_rolling['Time'], df_rolling['Membrane'],label='Membrane',color='orange',path_effects=[pe.Stroke(linewidth=3, foreground='black'), pe.Normal()])
    if pept_num != 0:
        ax.plot(df_rolling['Time'], df_rolling['Peptide'], label='Peptide',color='g',path_effects=[pe.Stroke(linewidth=3, foreground='black'), pe.Normal()])
        ax.plot(df['Time'],df['Peptide'],alpha=0.3,color='g')
    ax.plot(df['Time'],df['All'],alpha=0.3,color='b')
    ax.plot(df['Time'],df['Membrane'],alpha=0.3,color='orange')
    ax.set_xlabel("Time (ns)",fontsize=13)
    ax.set_ylabel(r"RMSD ($\AA$)",fontsize=13)
    # ax.set_ylim(-2.1911417040492087, 46.01398873468586)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.legend()
    plt.savefig(f'{plots_folder}1.RMSD/{dq_num}.{pept_num}.{repl_num}.svg')
    plt.close()
    

## Electron Density

In [21]:
def analyse_density(pept_folder,traj_file,dq_num,pept_num,repl_num):
    """ Analyse the density with GROMACS """
    if repl_num == 1: gmx_command_1 = f'(echo "POPC"; echo "POPC"; echo "TIP3"; echo "CLA"; echo "POT") | gmx density -f {traj_file} -s {pept_folder}step7_compel.tpr -n {pept_folder}index.ndx -ng 4 -o {data_folder}2.Density/{dq_num}.{pept_num}.{repl_num}.xvg -symm'
    else: gmx_command_1 = f'(echo "POPC"; echo "POPC"; echo "TIP3"; echo "CLA"; echo "POT") | gmx density -f {traj_file} -s {pept_folder}step7_compel_{repl_num}.tpr -n {pept_folder}index.ndx -ng 4 -o {data_folder}2.Density/{dq_num}.{pept_num}.{repl_num}.xvg -symm'
    subprocess.call(f'/bin/bash -c "source /usr/local/gromacs-2020.7/bin/GMXRC && {gmx_command_1}"', shell=True)

In [22]:
def plot_density(dq_num,pept_num,repl_num):
    """ Plot the density """
    df_dens = pd.read_csv(f'{data_folder}2.Density/{dq_num}.{pept_num}.{repl_num}.xvg',delimiter="\s+",skiprows=27,names=['Coordinate (nm)','POPC','TIP3','CLA','POT'])
    plt.plot(df_dens['Coordinate (nm)']*10,df_dens['POPC'],color='grey',label='Membrane',path_effects=[pe.Stroke(linewidth=3, foreground='black'), pe.Normal()])
    plt.plot(df_dens['Coordinate (nm)']*10,df_dens['TIP3'],color='cyan',label='Water',path_effects=[pe.Stroke(linewidth=3, foreground='black'), pe.Normal()])
    plt.plot(df_dens['Coordinate (nm)']*10,df_dens['POT'],label='K+',color='blue',path_effects=[pe.Stroke(linewidth=3, foreground='black'), pe.Normal()])    
    plt.plot(df_dens['Coordinate (nm)']*10,df_dens['CLA'],label='Cl-',color='red',path_effects=[pe.Stroke(linewidth=3, foreground='black'), pe.Normal()])
    plt.legend(bbox_to_anchor=(1,1))
    plt.xlabel('Box z coordinate ($\AA$)',fontsize=14)
    plt.ylabel('Density (kg·m$^{-3}$)',fontsize=14)
    plt.xticks(fontsize=13)
    plt.yticks(fontsize=14)
    plt.title('Partial density')
    plt.savefig(f'{plots_folder}2.Density/{dq_num}.{pept_num}.{repl_num}.svg')
    plt.close()

In [23]:
def analyse_density_per_membrane(pept_folder,traj_file,dq_num,pept_num,repl_num):
    """ This function analyses electron density using gromacs and differentiating between MEMB1 and MEMB2 """
    if repl_num == 1: gmx_command_1 = f'(echo "POPC"; echo "MEMB1"; echo "MEMB2"; echo "TIP3"; echo "CLA"; echo "POT") | gmx density -f {traj_file} -s {pept_folder}step7_compel.tpr -n {pept_folder}index.ndx -ng 6 -o {data_folder}2.Density/{dq_num}.{pept_num}.{repl_num}_2memb.xvg'
    else: gmx_command_1 = f'(echo "POPC"; echo "MEMB1"; echo "MEMB2"; echo "TIP3"; echo "CLA"; echo "POT") | gmx density -f {traj_file} -s {pept_folder}step7_compel_{repl_num}.tpr -n {pept_folder}index.ndx -ng 6 -o {data_folder}2.Density/{dq_num}.{pept_num}.{repl_num}_2memb.xvg'
    subprocess.call(f'/bin/bash -c "source /usr/local/gromacs-2020.7/bin/GMXRC && {gmx_command_1}"', shell=True)

In [24]:
def plot_density_per_membrane(dq_num,pept_num,repl_num):
    """ This function plots the electron density using gromacs and differentiating between MEMB1 and MEMB2 """
    df_dens = pd.read_csv(f'{data_folder}2.Density/{dq_num}.{pept_num}.{repl_num}_2memb.xvg',delimiter="\s+",skiprows=29,names=['Coordinate (nm)','POPC','MEMB1','MEMB2','TIP3','CLA','POT'])
    # plt.plot(df_dens['Coordinate (nm)']*10,df_dens['POPC'],color='grey',label='Membrane',path_effects=[pe.Stroke(linewidth=3, foreground='black'), pe.Normal()])
    plt.plot(df_dens['Coordinate (nm)']*10,df_dens['MEMB1'],color='orange',label='Membrane 1',path_effects=[pe.Stroke(linewidth=3, foreground='black'), pe.Normal()])
    plt.plot(df_dens['Coordinate (nm)']*10,df_dens['MEMB2'],color='purple',label='Membrane 2',path_effects=[pe.Stroke(linewidth=3, foreground='black'), pe.Normal()])
    plt.plot(df_dens['Coordinate (nm)']*10,df_dens['TIP3'],color='cyan',label='Water',path_effects=[pe.Stroke(linewidth=3, foreground='black'), pe.Normal()])
    plt.plot(df_dens['Coordinate (nm)']*10,df_dens['POT'],label='K+',color='blue',path_effects=[pe.Stroke(linewidth=3, foreground='black'), pe.Normal()])    
    plt.plot(df_dens['Coordinate (nm)']*10,df_dens['CLA'],label='Cl-',color='red',path_effects=[pe.Stroke(linewidth=3, foreground='black'), pe.Normal()])
    plt.legend(bbox_to_anchor=(1,1))
    plt.xlabel('Box z coordinate ($\AA$)',fontsize=14)
    plt.ylabel('Density (kg·m$^{-3}$)',fontsize=14)
    plt.xticks(fontsize=13)
    plt.yticks(fontsize=14)
    plt.title('Partial density')
    plt.savefig(f'{plots_folder}2.Density/{dq_num}.{pept_num}.{repl_num}_2memb.svg')
    plt.close()

## Pore 

In [28]:
def analyse_pore(u,pept_num,dq_num,repl_num,interacting_membrane='upper'):
    """ Analyse the pore for each simulation, but separately for each membrane (because it calculates the minimum pore for all Z-stacks, and we need to do it for one membrane at a time) """
    upper_membrane = u.select_atoms('resname POPC')[:34304] # There are 34304 atoms in each membrane
    lower_membrane = u.select_atoms('resname POPC')[34304:]
    #Getting only min radii of each frame w/ Numpy arrays
    Wpos = []
    for ts in u.trajectory:
        waters = u.select_atoms("resname TIP3 and name OH2")
        Wpos.append(waters.positions)

    Wpos = np.array(Wpos)

    #NUMPY ARRAYS --> x12 time faster
    pore_per_ts = [] #Here we will save, for each ts, the limiting point of the porus (min radii accross all Z stacks)

    #Iterate over trajectory positions X and Y of waters
    for ts in Wpos:
        pore_radiis = set() #Save all Z-stack por radii for each ts
        z_coordinates = [] # Save all membrane z positions in a list
        if interacting_membrane == 'upper':
            for position in upper_membrane.positions:
                z_coordinates.append(round(position[2]))
        else:
            for position in lower_membrane.positions:
                z_coordinates.append(round(position[2]))
        for i in range(min(z_coordinates),max(z_coordinates), 2): #Iterate over min to max Z positions (modificable) every 2 A
            slice_w = ts[(ts[...,2] > i) & (ts[...,2] < i+2)] #Slicing array by Z coordinate
            X = slice_w[:,0] # X positions
            Y = slice_w[:,1] # Y positions
            
            if len(X) == 0 or len(Y) == 0:
                WXOM = [0,0]
                pore_radiis.add(0)
            else:
                WCOM = [sum(X)/len(X), sum(Y)/len(Y)] #Center of mass of the waters
            
                #Distance between the WCOM and the furthest water from it
                #This is the radii of the circle that contains ALL waters for that stack
                pore_radiis.add(max([distance.euclidean(WCOM, [X[i],Y[i]]) for i,x in enumerate(X)]))
        #For each ts, add the radii of the smallest circle
        
        #This will be the Z stack with the smaller circle --> water pore bottleneck
        pore_per_ts.append(min(pore_radiis))
        
    df = pd.DataFrame()
    df['Time'] = [i/10 for i in range(len(pore_per_ts))]
    df['Pore'] = pore_per_ts
    df.to_csv(f'{data_folder}3.Pore/{dq_num}.{pept_num}.{repl_num}_{interacting_membrane}.csv')

In [31]:
def plot_pore(dq_num,pept_num,repl_num,sliding_window=10,interacting_membrane='upper'):
    """ Plot the pore for each simulation and membrane """
    df = pd.read_csv(f'{data_folder}3.Pore/{dq_num}.{pept_num}.{repl_num}_{interacting_membrane}.csv',index_col=0).rolling(sliding_window).mean()
    plt.plot(df['Pore'],color=pept_colors[pept_num],path_effects=[pe.Stroke(linewidth=3, foreground='black'), pe.Normal()])
    plt.xlabel('Time (ns)')
    plt.ylabel('Radius ($\AA$)')
    plt.title(f'{peptide_names[pept_num]} pore radius distribution')
    plt.savefig(f'{plots_folder}3.Pore/{dq_num}.{pept_num}.{repl_num}_{interacting_membrane}.svg')
    plt.close()

In [36]:
def plot_all_mean_pore(dq_num):
    """ This function plots the average pore size plot in each dq_num for each peptide """
    fig,ax = plt.subplots(figsize=(17,10))
    for pept_num in range(len(peptide_names)):
        df1 = pd.read_csv(f'{data_folder}3.Pore/{dq_num}.{pept_num}.1_upper.csv').rolling(10).mean()
        df2 = pd.read_csv(f'{data_folder}3.Pore/{dq_num}.{pept_num}.1_lower.csv').rolling(10).mean()
        df3 = pd.read_csv(f'{data_folder}3.Pore/{dq_num}.{pept_num}.2_upper.csv').rolling(10).mean()
        df4 = pd.read_csv(f'{data_folder}3.Pore/{dq_num}.{pept_num}.2_lower.csv').rolling(10).mean()
        # Calculate the membrane with the pore. This generates a different df for each peptide
        df = pd.DataFrame()
        df['Unnamed: 0'] = df3['Unnamed: 0']
        df['Time'] = df3['Time']
        if df1['Pore'].mean() > df2['Pore'].mean(): df['Replica1'] = df1['Pore']
        else: df['Replica1'] = df2['Pore']
        if df3['Pore'].mean() > df4['Pore'].mean(): df['Replica2'] = df3['Pore']
        else: df['Replica2'] = df4['Pore']
        # Only the dq_num == 4 has 3 replicas
        if dq_num == 4:
            df5 = pd.read_csv(f'{data_folder}3.Pore/{dq_num}.{pept_num}.3_upper.csv',index_col=0).rolling(10).mean()
            df6 = pd.read_csv(f'{data_folder}3.Pore/{dq_num}.{pept_num}.3_lower.csv',index_col=0).rolling(10).mean()
            if df5['Pore'].mean() > df6['Pore'].mean(): df['Replica3'] = df5['Pore']
            else: df['Replica3'] = df6['Pore']
            df['Mean'] = df[['Replica1','Replica2','Replica3']].mean(axis=1)
        else: df['Mean'] = df[['Replica1','Replica2']].mean(axis=1)
        if pept_num != 0 and dq_num != 2: plt.plot(df['Unnamed: 0'],df['Mean'],color=pept_colors[pept_num],path_effects=[pe.Stroke(linewidth=3, foreground='black'), pe.Normal()],label=peptide_names[pept_num])
        else: plt.plot(df['Unnamed: 0'],df['Mean'],color=pept_colors[pept_num],path_effects=[pe.Stroke(linewidth=3, foreground='black'), pe.Normal()],label=peptide_names[pept_num])
        # This is to print the mean pore size for each peptide (for replica and the total average)
        print(df['Replica1'].mean(),df['Replica2'].mean(),df['Replica3'].mean(),df['Mean'].mean())
    plt.xlabel('Time (ns)')
    plt.ylabel('Pore length ($\AA$)')
    plt.legend(bbox_to_anchor=(1,1))
    # plt.savefig(f'{plots_folder}3.Pore/all_mean_{dq_num}.svg',transparent=True)
    plt.show()
    

In [38]:
def plot_all_pore(dq_num,repl_num):
    """ This function plots the average plot for each dq_num, pept_num and repl_num """
    for pept_num in range(len(peptide_names)):
        df1 = pd.read_csv(f'{data_folder}3.Pore/{dq_num}.{pept_num}.{repl_num}_upper.csv',index_col=0).rolling(10).mean()
        df2 = pd.read_csv(f'{data_folder}3.Pore/{dq_num}.{pept_num}.{repl_num}_lower.csv',index_col=0).rolling(10).mean()
        # Check which of the two membranes has a pore
        if df1['Pore'].mean() > df2['Pore'].mean(): df = df1
        else: df = df2
        # print(df['Pore'].mean())
        plt.plot(df['Pore'],color=pept_colors[pept_num],path_effects=[pe.Stroke(linewidth=3, foreground='black'), pe.Normal()],label=peptide_names[pept_num])
    plt.xlabel('Time (ns)')
    plt.ylabel('Pore length ($\AA$)')
    plt.legend(bbox_to_anchor=(1,1))
    plt.savefig(f'{plots_folder}3.Pore/all_{dq_num}.{repl_num}.svg',transparent=True)
    plt.show()

## Lipid parameters

In [14]:
def analyse_lipid_order_parameters(u,dq_num,pept_num,repl_num,lipid='POPC'):
    """ Here we calculate the lipid order parameters for POPC, but we focus on sn-1 (palmitoyl) segment """
    # Create an empty DataFrame to store lipid order parameters
    df_lop = pd.DataFrame()
    lop = []  # List to store order parameters
    
    # Iterate over carbon numbers 3 to 17
    for c_number in range(3, 17):
        # Define selection criteria for atoms
        selection = f'resname {lipid} and name C3{c_number} H{c_number}X H{c_number}Y'
        # Select atoms based on the selection criteria
        atom_selection = u.select_atoms(selection)
        # Extract trajectory data (positions) of selected atoms
        skip = len(set(atom_selection.names))
        data = u.trajectory.timeseries(atom_selection)
        # Transpose the trajectory data so that its structure is (atoms,frames,positions)
        data2 = np.transpose(data, (1, 0, 2))
        # Calculate displacement vectors between frames
        conc_data = [data2[n::skip]-data2[0::skip] for n in range(1,skip)]
        cd = np.concatenate(conc_data, axis=0) #Then concatenate
        #     cd = np.concatenate((data2[1::3] - data2[0::3], data2[2::3] - data2[0::3]), axis=0)
        # Calculate magnitude of displacement vectors
        cd_r = np.sqrt(np.sum(np.power(cd, 2), axis=-1))
        # Calculate cosine of the angle between displacement vectors and z-axis
        cos_theta = cd[..., 2] / cd_r
        # Calculate lipid order parameter using the formula
        s_cd = -0.5 * (3. * np.square(cos_theta) - 1)
        # Reshape the array to appropriate dimensions
        # s_cd.shape = (int(s_cd.shape[0]/(skip-1)), s_cd.shape[1], -1)
        # Calculate average order parameter
        order_param = np.average(s_cd)
        # order_param = np.std(s_cd) # To calculate std
        # Append the order parameter to the list
        lop.append(order_param)
    
    # Create a list of carbon numbers
    c_numbers = [i for i in range(3, 17)]
    # Add carbon numbers and order parameters to the DataFrame
    df_lop['Cn'] = c_numbers
    df_lop['LOP'] = lop
    # Save DataFrame to a CSV file
    df_lop.to_csv(f'{data_folder}4.LipidParameters/{dq_num}.{pept_num}.{repl_num}_lop.csv')
    # df_lop.to_csv(f'{data_folder}5.LipidParameters/{pept_num}.{memb_num}_lop_{lipid.lower()}_std.csv')


def plot_lipid_order_parameters(dq_num,pept_num,repl_num):
    df_lop = pd.read_csv(f'{data_folder}4.LipidParameters/{dq_num}.{pept_num}.{repl_num}_lop.csv')
    plt.plot(df_lop['Cn'],df_lop['LOP'],color=pept_colors[pept_num],label=peptide_names[pept_num])
    plt.scatter(df_lop['Cn'],df_lop['LOP'],color=pept_colors[pept_num],facecolors='None') # Add also points to the graph (empty, only circunferences)
    plt.xlabel('Carbon number')
    plt.ylabel('SCD')
    plt.savefig(f'{plots_folder}4.LipidParameters/{dq_num}.{pept_num}.{repl_num}_lop.svg',transparent=True)
    plt.show()

In [15]:
def plot_avg_lipid_order_parameters(pept_num,dq_num=4):
    df_lop_1 = pd.read_csv(f'{data_folder}4.LipidParameters/{dq_num}.{pept_num}.1_lop.csv')
    df_lop_2 = pd.read_csv(f'{data_folder}4.LipidParameters/{dq_num}.{pept_num}.2_lop.csv')
    df_lop_3 = pd.read_csv(f'{data_folder}4.LipidParameters/{dq_num}.{pept_num}.3_lop.csv')
    df_lop = pd.DataFrame()
    df_lop['Cn'] = df_lop_1['Cn']
    df_lop['LOP1'] = df_lop_1['LOP']
    df_lop['LOP2'] = df_lop_2['LOP']
    df_lop['LOP3'] = df_lop_3['LOP']
    df_lop['LOP'] = df_lop[['LOP1','LOP2','LOP3']].mean(axis=1)
    plt.figure()
    plt.plot(df_lop['Cn'],df_lop['LOP'],color=pept_colors[pept_num],label=peptide_names[pept_num])
    plt.scatter(df_lop['Cn'],df_lop['LOP'],color=pept_colors[pept_num],facecolors='None') # Add also points to the graph (empty, only circunferences)
    # plt.xlabel('Carbon number',fontsize=14)
    # plt.ylabel('SCD',fontsize=14)
    plt.xticks(fontsize=13)
    plt.yticks(fontsize=13)
    plt.ylim((0.014588689383333336, 0.18680707161666665))
    plt.savefig(f'{plots_folder}4.LipidParameters/{dq_num}.{pept_num}.avg_lop.svg',transparent=True)
    plt.close()

## Secondary Structure

### Tcl

In [56]:
def ss_tcl_csv(dq_num,pept_num,repl_num):
    """ This function modifies the file generated with vmd_ss.tcl to create a csv file, which is then used for the plotting function. Thus, this function needs that the vmd_ss.tcl script has been previously run. """
    with open(f'{data_folder}5.SS/{dq_num}.{pept_num}.{repl_num}.txt') as f:
        data = []
        for line in f:
            if line[0] != 'F':
                parts = line.split()
                frame = int(parts[0])
                residue = int(parts[1])
                ss = parts[-1]
                data.append({'Frame':frame,'Residue':residue,'Secondary_Structure':ss})
    df = pd.DataFrame(data)
    df.to_csv(f'{data_folder}5.SS/{dq_num}.{pept_num}.{repl_num}.csv')

In [57]:
def plot_ss_csv(dq_num,pept_num,repl_num):
    df = pd.read_csv(f'{data_folder}5.SS/{dq_num}.{pept_num}.{repl_num}.csv',index_col=0)
    df['Residue'] = df['Residue'] -df['Residue'][0]
    # Map secondary structure strings to numbers
    structure_mapping = {'B': 0, 'E': 1, 'I': 2, 'H': 3, 'G': 4, 'T': 5, 'C': 6}
    structure_names = {
        'B': 'β Bridge', 
        'E': 'β Sheet', 
        'I': 'π Helix', 
        'H': 'α Helix', 
        'G': '3₁₀ Helix', 
        'T': 'Turn', 
        'C': 'Coil'
    }

    # Apply mapping to the DataFrame
    df['Secondary_Structure'] = df['Secondary_Structure'].map(structure_mapping)

    # Determine the number of frames and residues for the 2D array
    num_frames = df['Frame'].nunique()
    num_residues = df['Residue'].nunique()

    # Create a 2D array for plotting
    data_array = np.full((num_frames, num_residues), fill_value=-1)  # Initialize with -1 for missing values

    # Populate the 2D array with secondary structure data
    for _, row in df.iterrows():
        frame = int(row['Frame'])
        residue = int(row['Residue'])  # Adjust residue index to start from 0
        ss_value = row['Secondary_Structure']
        data_array[frame, residue] = ss_value

    # Transpose the 2D array to have frames (time) on the x-axis and residues on the y-axis
    data_array = data_array.T

    # Define colors for each structure type as specified
    # colors = ['#4A90E2', '#007ACC', '#D33682', '#9B59B6', '#6A0DAD', '#FFD700', '#FFA500']
    colors = [
        # "#660066",  # Deep purple
        "#993399",  # Dark purple
        "#cc33cc",  # Magenta
        "#ff6600",  # Dark orange
        "#ff9933",  # Orange
        "#ffcc99",  # Soft orange
        "#ffe5cc",  # Light orange
        "#ffffcc"   # Pale yellow
    ]
    cmap = mcolors.ListedColormap(colors)
    norm = mcolors.BoundaryNorm(boundaries=np.arange(-0.5, len(structure_mapping) + 0.5, 1), ncolors=len(colors))

    # Plot the data with the transposed array
    plt.figure(figsize=(12, 8))
    plt.imshow(data_array, aspect='auto', cmap=cmap, interpolation='nearest')

    # Add colorbar with structure names
    cbar = plt.colorbar(ticks=range(len(structure_names)), label="Secondary Structure")
    cbar.ax.set_yticklabels(list(structure_names.values()))

    # Set labels
    plt.xlabel("Time (ns)",fontsize=14)
    plt.ylabel("Residue",fontsize=14)
    # plt.title("Secondary Structure Over Time")
    
    # Select every 9th residue label and corresponding index
    y_ticks = range(0, len(all_residue_labels[pept_num]))
    # Set y-axis ticks and labels for residues
    plt.yticks(y_ticks, labels=all_residue_labels[pept_num],fontsize=14)
    # Save and close the plot
    plt.savefig(f'{plots_folder}5.SS/{dq_num}.{pept_num}.{repl_num}.svg',transparent=True)
    plt.close()

In [58]:
def plot_avg_ss_csv(dq_num,pept_num):
    """ Plot the average secondary structure for each peptide """
    df1 = pd.read_csv(f'{data_folder}5.SS/{dq_num}.{pept_num}.1.csv',index_col=0)
    df2 = pd.read_csv(f'{data_folder}5.SS/{dq_num}.{pept_num}.2.csv',index_col=0)
    df3 = pd.read_csv(f'{data_folder}5.SS/{dq_num}.{pept_num}.3.csv',index_col=0)
    # Combine into same df
    combined_df = pd.concat([df1[['Frame', 'Residue', 'Secondary_Structure']],
                            df2['Secondary_Structure'],
                            df3['Secondary_Structure']],
                            axis=1)
    # Rename columns for clarity
    combined_df.columns = ['Frame', 'Residue', 'SS_1', 'SS_2', 'SS_3']

    # Calculate the mode (most common value) row-wise for the Secondary_Structure columns
    combined_df['Secondary_Structure'] = combined_df[['SS_1', 'SS_2', 'SS_3']].mode(axis=1)[0]
    # Select the final dataframe
    df = combined_df[['Frame', 'Residue', 'Secondary_Structure']].copy()
    # All residue numbers have to start by 1
    df['Residue'] = df['Residue'] -df['Residue'].iloc[0]
    # Map secondary structure strings to numbers
    structure_mapping = {'B': 0, 'E': 1, 'I': 2, 'H': 3, 'G': 4, 'T': 5, 'C': 6}
    structure_names = {
        'B': 'β Bridge', 
        'E': 'β Sheet', 
        'I': 'π Helix', 
        'H': 'α Helix', 
        'G': '3₁₀ Helix', 
        'T': 'Turn', 
        'C': 'Coil'
    }

    # Apply mapping to the DataFrame
    df['Secondary_Structure'] = df['Secondary_Structure'].map(structure_mapping)

    # Determine the number of frames and residues for the 2D array
    num_frames = df['Frame'].nunique()
    num_residues = df['Residue'].nunique()

    # Create a 2D array for plotting
    data_array = np.full((num_frames, num_residues), fill_value=-1)  # Initialize with -1 for missing values

    # Populate the 2D array with secondary structure data
    for _, row in df.iterrows():
        frame = int(row['Frame'])
        residue = int(row['Residue'])  # Adjust residue index to start from 0
        ss_value = row['Secondary_Structure']
        data_array[frame, residue] = ss_value

    # Transpose the 2D array to have frames (time) on the x-axis and residues on the y-axis
    data_array = data_array.T

    # Define colors for each structure type as specified
    # colors = ['#4A90E2', '#007ACC', '#D33682', '#9B59B6', '#6A0DAD', '#FFD700', '#FFA500']
    colors = [
        # "#660066",  # Deep purple
        "#993399",  # Dark purple
        "#cc33cc",  # Magenta
        "#ff6600",  # Dark orange
        "#ff9933",  # Orange
        "#ffcc99",  # Soft orange
        "#ffe5cc",  # Light orange
        "#ffffcc"   # Pale yellow
    ]
    cmap = mcolors.ListedColormap(colors)
    norm = mcolors.BoundaryNorm(boundaries=np.arange(-0.5, len(structure_mapping) + 0.5, 1), ncolors=len(colors))

    # Plot the data with the transposed array
    plt.figure(figsize=(12, 8))
    plt.imshow(data_array, aspect='auto', cmap=cmap, interpolation='nearest')

    # Add colorbar with structure names
    cbar = plt.colorbar(ticks=range(len(structure_names)), label="Secondary Structure")
    cbar.ax.set_yticklabels(list(structure_names.values()))

    # Set labels
    # plt.xlabel("Time (ns)",fontsize=14)
    # plt.ylabel("Residue",fontsize=14)
    # plt.title("Secondary Structure Over Time")
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=25)
    
    # Select every 9th residue label and corresponding index
    y_ticks = range(0, len(all_residue_labels[pept_num]))
    # Set y-axis ticks and labels for residues
    plt.yticks(y_ticks, labels=all_residue_labels[pept_num],fontsize=14)
    # Save and close the plot
    plt.savefig(f'{plots_folder}5.SS/{dq_num}.{pept_num}_avg.svg',transparent=True)
    plt.close()


## Occupancy

In [73]:
def analyse_occupancy_lipid_part(u,traj_file,parm_file,lipid_part,dq_num,pept_num,repl_num):
    """ This function calculates the occupancy of the residues in the upper and lower leaflets of the membrane """
    folder = f'{data_folder}6.Occupancy/{peptide_names[pept_num]}'
    # Define atoms of the correspondent lipid part
    if lipid_part == 'head': lipid_atoms = head_atoms
    else: lipid_atoms = tail_atoms

    resnames_long = list(u.select_atoms('protein').residues.resnames)
    resnames_short = [mda.lib.util.convert_aa_code(i) for i in resnames_long]
    lipid = 'POPC'
    cutoffs = [0.35]
    nprot = 1
    timeunit = 'ns'

    li = LipidInteraction(traj_file, topfile_list=parm_file, cutoffs=cutoffs, lipid=lipid,
                    nprot=nprot, save_dir=folder,lipid_atoms=lipid_atoms)
    li.collect_residue_contacts()
    # Create array of residue length and simulation duration
    upper_contacts = [[[[] for j in range(len(u.trajectory))]] for i in range(peptides_length[pept_num])]
    lower_contacts = [[[[] for j in range(len(u.trajectory))]] for i in range(peptides_length[pept_num])]

    # Add the lipid numbers of interaction in the upper_ or lower_contacts arrays in the correct residue and frame
    for residue in range(peptides_length[pept_num]):
        for frame,contacts in enumerate(li._contact_residues_high[residue][0]):
            for lipid_number in contacts:
                if lipid_number < 128 or lipid_number > 256 and lipid_number < 384: #Lipid numbers 0 to 128 are upper leaflet, 256 to  lower leaflet
                    lower_contacts[residue][0][frame].append(lipid_number)
                else:
                    upper_contacts[residue][0][frame].append(lipid_number)
    # Compute occupancies
    upper_occupancies = []
    for residue in range(peptides_length[pept_num]):
        residue_i_contacts = []
        for i in range(len(u.trajectory)):
            if len(upper_contacts[residue][0][i]) > 0: # If there is at least one lipid interacting, we add 1 (since we want to see whether there is interaction or not)
                residue_i_contacts.append(1)
            else: residue_i_contacts.append(0) # If no lipid interacting, we add 0
        upper_occupancies.append(sum(residue_i_contacts)/len(u.trajectory)*100) # Sum and divide by trajectory length and multiply by 100 to get the percentage of occupancy per residue

    lower_occupancies = []
    for residue in range(peptides_length[pept_num]):
        residue_i_contacts = []
        for i in range(len(u.trajectory)):
            if len(lower_contacts[residue][0][i]) > 0:
                residue_i_contacts.append(1)
            else: residue_i_contacts.append(0)
        lower_occupancies.append(sum(residue_i_contacts)/len(u.trajectory)*100)

    df_upper = pd.DataFrame()
    df_upper['Resnames'] = resnames_short
    df_upper['Occupancies'] = upper_occupancies
    df_upper.to_csv(f'{data_folder}6.Occupancy/{dq_num}.{pept_num}.{repl_num}_{lipid_part}_upper.csv')

    df_lower = pd.DataFrame()
    df_lower['Resnames'] = resnames_short
    df_lower['Occupancies'] = lower_occupancies
    df_lower.to_csv(f'{data_folder}6.Occupancy/{dq_num}.{pept_num}.{repl_num}_{lipid_part}_lower.csv')


In [74]:
def average_occupancies(dq_num,pept_num,part):
    """ This function averages the occupancies of the residues in the upper and lower leaflets of the membrane """
    if dq_num == 4: repl_nums = [1,2,3]
    else: repl_nums = [1,2]
    df_avg = pd.DataFrame()
    df_avg['Resnames'] = pd.read_csv(f'{data_folder}6.Occupancy/{dq_num}.{pept_num}.1_{part}.csv',usecols=['Resnames'])
    for repl_num in repl_nums:
        df = pd.read_csv(f'{data_folder}6.Occupancy/{dq_num}.{pept_num}.{repl_num}_{part}.csv',index_col=0)
        df_avg[f'R{repl_num}'] = df['Occupancies']
    df_avg['Mean'] = df_avg[[f'R{repl_num}' for repl_num in repl_nums]].mean(axis=1)
    df_avg.to_csv(f'{data_folder}6.Occupancy/avg_{dq_num}.{pept_num}_{part}.csv')

In [75]:
def average_occupancies_lipid_part(dq_num,pept_num,part,lipid_part):
    """ This function averages the occupancies of the residues in the upper and lower leaflets of the membrane differentiating between head and tail """
    if dq_num == 4: repl_nums = [1,2,3]
    else: repl_nums = [1,2]
    df_avg = pd.DataFrame()
    df_avg['Resnames'] = pd.read_csv(f'{data_folder}6.Occupancy/{dq_num}.{pept_num}.1_{lipid_part}_{part}.csv',usecols=['Resnames'])
    for repl_num in repl_nums:
        df = pd.read_csv(f'{data_folder}6.Occupancy/{dq_num}.{pept_num}.{repl_num}_{lipid_part}_{part}.csv',index_col=0)
        df_avg[f'R{repl_num}'] = df['Occupancies']
    df_avg['Mean'] = df_avg[[f'R{repl_num}' for repl_num in repl_nums]].mean(axis=1)
    df_avg.to_csv(f'{data_folder}6.Occupancy/avg_{dq_num}.{pept_num}_{lipid_part}_{part}.csv')

In [77]:
def plot_occupancy_lipid_part(dq_num,pept_num,repl_num,lipid_part,part):
    """ This function plots the occupancy of the residues in the upper and lower leaflets of the membrane """
    peptide_name = peptide_names[pept_num]
    peptide_color = pept_colors[pept_num]
    df = pd.read_csv(f'{data_folder}6.Occupancy/{dq_num}.{pept_num}.{repl_num}_{lipid_part}_{part}.csv',index_col=0)
    # ------- PART 1: Create background
 
    # number of variable
    categories=list(df['Resnames'])
    N = len(categories)
    # What will be the angle of each axis in the plot? (we divide the plot / number of variable)
    angles = [n / float(N) * 2 * 3.14 for n in range(N)]
    angles += angles[:1]
    # Initialise the spider plot
    ax = plt.subplot(111, polar=True)
    # If you want the first axis to be on top:
    ax.set_theta_offset(3.14 / 2)
    ax.set_theta_direction(-1)
    # Draw one axe per variable + add labels
    plt.xticks(angles[:-1], categories, size=15, color=peptide_color, weight='bold')
    plt.tick_params(axis="x", which="major", pad=25)
    # Draw ylabels
    ax.set_rlabel_position(0)
    plt.yticks([20,40,60,80,100], ["20%","40%","60%","80%","100%"], color="grey", size=15)
    plt.ylim(0,100)
    

    # ------- PART 2: Add plots

    # Plot each individual = each line of the data
    # I don't make a loop, because plotting more than 3 groups makes the chart unreadable

    # Ind1
    values=df['Occupancies'].values.flatten().tolist()
    values += values[:1]
    ax.plot(angles, values, linewidth=3, linestyle='solid', label=f'{peptide_name}', color=peptide_color)
    ax.fill(angles, values, alpha=0.2, color=peptide_color)
    plt.xticks(angles[:-1], categories, size=15, color=peptide_color, weight='bold')
    plt.tick_params(axis="x", which="major", pad=25)

    # Add legend
    # plt.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))
    plt.rcParams["figure.figsize"] = (10,10)

    # Show the graph
    plt.savefig(f'{plots_folder}6.Occupancy/{dq_num}.{pept_num}.{repl_num}_{lipid_part}_{part}.svg', bbox_inches='tight')
    plt.close()

In [78]:
def plot_avg_occupancy_lipid_part(dq_num,pept_num,part,lipid_part):
    """ This function plots the average occupancy of the residues in the upper and lower leaflets of the membrane """
    peptide_name = peptide_names[pept_num]
    peptide_color = pept_colors[pept_num]
    df = pd.read_csv(f'{data_folder}6.Occupancy/avg_{dq_num}.{pept_num}_{lipid_part}_{part}.csv',index_col=0)
    # ------- PART 1: Create background
 
    # number of variable
    categories=list(df['Resnames'])
    N = len(categories)
    # What will be the angle of each axis in the plot? (we divide the plot / number of variable)
    angles = [n / float(N) * 2 * 3.14 for n in range(N)]
    angles += angles[:1]
    # Initialise the spider plot
    ax = plt.subplot(111, polar=True)
    # If you want the first axis to be on top:
    ax.set_theta_offset(3.14 / 2)
    ax.set_theta_direction(-1)
    # Draw one axe per variable + add labels
    plt.xticks(angles[:-1], categories, size=25, color=peptide_color, weight='bold')
    plt.tick_params(axis="x", which="major", pad=25)
    # Draw ylabels
    ax.set_rlabel_position(0)
    plt.yticks([20,40,60,80,100], ["20%","40%","60%","80%","100%"], color="grey", size=25)
    plt.ylim(0,100)
    

    # ------- PART 2: Add plots

    # Plot each individual = each line of the data
    # I don't make a loop, because plotting more than 3 groups makes the chart unreadable

    # Ind1
    values=df['Mean'].values.flatten().tolist()
    values += values[:1]
    ax.plot(angles, values, linewidth=3, linestyle='solid', label=f'{peptide_name}', color=peptide_color)
    ax.fill(angles, values, alpha=0.2, color=peptide_color)
    plt.xticks(angles[:-1], categories, size=35, color=peptide_color, weight='bold')
    plt.tick_params(axis="x", which="major", pad=35)

    # Add legend
    # plt.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))
    plt.rcParams["figure.figsize"] = (10,10)

    # Show the graph
    plt.savefig(f'{plots_folder}6.Occupancy/avg_{dq_num}.{pept_num}_{lipid_part}_{part}.svg', bbox_inches='tight')
    plt.close()

## Potentials

In [86]:
def analyse_potentials(dq_num,pept_num,repl_num,pept_folder,center_traj):
    """ Analyse the potentials with gromacs """
    gmx_command = f'(echo "System") | gmx potential -f {center_traj} -s {pept_folder}step7_compel.tpr -n {pept_folder}index.ndx -o {data_folder}7.Potentials/{dq_num}.{pept_num}.{repl_num}_potential.xvg -oc {data_folder}7.Potentials/{dq_num}.{pept_num}.{repl_num}_charge.xvg -of {data_folder}7.Potentials/{dq_num}.{pept_num}.{repl_num}_field.xvg -sl 14'
    subprocess.call(f'/bin/bash -c "source /usr/local/gromacs-2020.7/bin/GMXRC && {gmx_command}"', shell=True)
    for magnitude in ['potential','charge','field']:
        box = []
        values = []
        with open(f'{data_folder}7.Potentials/{dq_num}.{pept_num}.{repl_num}_{magnitude}.xvg') as f:
            for line in f:
                if line.startswith('#') or line.startswith('@'):
                    continue
                else:
                    parts = line.split()
                    box.append(float(parts[0]))
                    values.append(float(parts[1]))
        df = pd.DataFrame()
        df['Box'] = box
        df[f'{magnitude}'] = values
        df.to_csv(f'{data_folder}7.Potentials/{dq_num}.{pept_num}.{repl_num}_{magnitude}.csv')

In [87]:
def analyse_potentials_symm(dq_num,pept_num,repl_num,pept_folder,center_traj):
    """ Analyse the potentials with gromacs using the symmetry option """
    gmx_command = f'(echo "System"; echo "POPC") | gmx potential -f {center_traj} -s {pept_folder}step7_compel.tpr -n {pept_folder}index.ndx -o {data_folder}7.Potentials/{dq_num}.{pept_num}.{repl_num}_potential.xvg -oc {data_folder}7.Potentials/{dq_num}.{pept_num}.{repl_num}_charge.xvg -of {data_folder}7.Potentials/{dq_num}.{pept_num}.{repl_num}_field.xvg -sl 15 -symm -center'
    # gmx_command = f'(echo "POPC"; echo "POPC") | gmx potential -f {center_traj} -s {pept_folder}step7_compel.tpr -n {pept_folder}index.ndx -o {data_folder}7.Potentials/{dq_num}.{pept_num}.{repl_num}_potential.xvg -sl 15 -symm -center'
    subprocess.call(f'/bin/bash -c "source /usr/local/gromacs-2023.1/bin/GMXRC && {gmx_command}"', shell=True)
    for magnitude in ['potential']:
        box = []
        values = []
        with open(f'{data_folder}7.Potentials/{dq_num}.{pept_num}.{repl_num}_{magnitude}.xvg') as f:
            for line in f:
                if line.startswith('#') or line.startswith('@'):
                    continue
                else:
                    parts = line.split()
                    box.append(float(parts[0]))
                    values.append(float(parts[1]))
        df = pd.DataFrame()
        df['Box'] = box
        df[f'{magnitude}'] = values
        df.to_csv(f'{data_folder}7.Potentials/{dq_num}.{pept_num}.{repl_num}_{magnitude}.csv')

In [88]:
def plot_potentials_symm(dq_num,pept_num,repl_num):
    """ Plot the potential, charge and field for each simulation but using the symmetry option when analysing with gromacs """
    for magnitude in ['potential']:
        df = pd.read_csv(f'{data_folder}7.Potentials/{dq_num}.{pept_num}.{repl_num}_{magnitude}.csv',index_col=0)
        plt.plot(df['Box'],df[f'{magnitude}'],path_effects=[pe.Stroke(linewidth=3, foreground='black'), pe.Normal()],color=pept_colors[pept_num])
        # plt.xlim(0,15)
        # plt.ylim(-0.95,1.45)
        plt.xlabel('Box (nm)')
        if magnitude == 'potential': plt.ylabel('Potential (V)')
        elif magnitude == 'charge': plt.ylabel('Charge density (q/nm$^{3}$)')
        else: plt.ylabel('Field (V/nm)')
        plt.title(f'{magnitude.capitalize()} Across the Box')
        plt.savefig(f'{plots_folder}7.Potentials/{dq_num}.{pept_num}.{repl_num}_{magnitude}.svg')
        plt.close()

In [90]:
def plot_potentials(dq_num,pept_num,repl_num):
    """ Plot the potential, charge and field for each simulation """
    for magnitude in ['potential','charge','field']:
        df = pd.read_csv(f'{data_folder}7.Potentials/{dq_num}.{pept_num}.{repl_num}_{magnitude}.csv',index_col=0)
        plt.plot(df['Box'],df[f'{magnitude}'],path_effects=[pe.Stroke(linewidth=3, foreground='black'), pe.Normal()],color=pept_colors[pept_num])
        # plt.xlim(0,15)
        # plt.ylim(0,5)
        plt.xlabel('Box (nm)')
        if magnitude == 'potential': plt.ylabel('Potential (V)')
        elif magnitude == 'charge': plt.ylabel('Charge density (q/nm$^{3}$)')
        else: plt.ylabel('Field (V/nm)')
        plt.title(f'{peptide_names[pept_num]} {magnitude.capitalize()} Across the Box')
        plt.savefig(f'{plots_folder}7.Potentials/{dq_num}.{pept_num}.{repl_num}_{magnitude}.svg')
        plt.close()

## Outcomes

In [93]:
def behavior():
    """ This function plots the outcomes for each peptide. The outcomes are determined by visual inspection with VMD """
    behaviours_data = {
        'dq_0': {'Partitioning': 10, 'Insertion': 0, 'Translocation': 0},
        'dq_8': {'Partitioning': 10, 'Insertion': 0, 'Translocation': 0},
        'dq_12': {'Partitioning': 8, 'Insertion': 2, 'Translocation': 0},
        'dq_16': {'Partitioning': 5, 'Insertion': 6, 'Translocation': 4},
        'dq_24': {'Partitioning': 1, 'Insertion': 2, 'Translocation': 7}
    }

    # Colors for the pie chart segments
    colors = ['#66b3ff', 'orchid', 'orange']

    # Function to format percentages and hide zeros
    def autopct_format(pct):
        return ('%1.0f%%' % pct) if pct > 0 else ''

    # Plotting pie charts for each dictionary
    fig, axes = plt.subplots(1, 5, figsize=(17, 18))

    for i, (key, values) in enumerate(behaviours_data.items()):
        totals = list(values.values())
        
        # Plot each pie chart without labels
        wedges, texts, autotexts = axes[i].pie(totals, autopct=autopct_format, startangle=90, colors=colors, wedgeprops = { 'linewidth' : 2, 'edgecolor' : 'k' })
        for autotext in autotexts: autotext.set_fontsize(14)
        # axes[i].set_title(f'DQ {key[3:]}',fontsize=16)
        # Add a white circle in the center to create a donut effect
        center_circle = plt.Circle((0, 0), 0.40, color='k', fc='k')
        axes[i].add_artist(center_circle)
        
        # Add the title inside the circle
        axes[i].text(0, 0, f'∆Q {key[3:]}', horizontalalignment='center', verticalalignment='center', fontsize=12, weight='bold',color='w')


    # Add a shared legend for all pie charts
    fig.legend(wedges, ['Partitioning', 'Insertion', 'Translocation'], loc='center right', bbox_to_anchor=(1.1, 0.5),fontsize=12)

    # Equal aspect ratio ensures that pies are drawn as circles
    plt.tight_layout()  # Adjust layout to fit the legend
    plt.savefig(f'{plots_folder}behaviours.svg')
    plt.show()

In [94]:
def behavior_16dq():
    """ This function plots the outcomes for each peptide in the 16dq system"""
    behaviours_16dq = {
        'Arg9': {'Partitioning': 2, 'Insertion': 0, 'Translocation': 1},
        'MAP': {'Partitioning': 2, 'Insertion': 0, 'Translocation': 1},
        'TP10': {'Partitioning': 0, 'Insertion': 1, 'Translocation': 2},
        'TP2': {'Partitioning': 1, 'Insertion': 2, 'Translocation': 0},
        'Leu9': {'Partitioning': 0, 'Insertion': 3, 'Translocation': 0}
    }

    # Colors for the pie chart segments
    colors = ['#66b3ff', 'orchid', 'orange']

    # Function to format percentages and hide zeros
    def autopct_format(pct):
        return ('%1.0f%%' % pct) if pct > 0 else ''

    # Plotting pie charts for each dictionary
    fig, axes = plt.subplots(1, 5, figsize=(17, 18))

    for i, (peptide, values) in enumerate(behaviours_16dq.items()):
        totals = list(values.values())
        
        # Plot each pie chart without labels
        wedges, texts, autotexts = axes[i].pie(totals, autopct=autopct_format, startangle=90, colors=colors, wedgeprops = { 'linewidth' : 2, 'edgecolor' : 'k' })
        for autotext in autotexts: autotext.set_fontsize(14)
        # Add a black circle in the center to create a donut effect
        center_circle = plt.Circle((0, 0), 0.40, color='k', fc='k')
        axes[i].add_artist(center_circle)
        
        # Add the title inside the circle
        axes[i].text(0, 0, peptide, horizontalalignment='center', verticalalignment='center', fontsize=12, weight='bold',color='w')


    # Add a shared legend for all pie charts
    fig.legend(wedges, ['Partitioning', 'Insertion', 'Translocation'], loc='center right', bbox_to_anchor=(1.1, 0.5),fontsize=12)

    # Equal aspect ratio ensures that pies are drawn as circles
    plt.tight_layout()  # Adjust layout to fit the legend
    plt.savefig(f'{plots_folder}behaviours_16dq.svg')
    plt.show()

In [95]:
def behaviour_8peptides_16dq():
    """ This function plots the behaviours of the 8 peptides in the sum of the 3 replicas. Need to change the negative value in the x axis to positive with Inkscape"""
    behaviours = {'Arg9':[3,0], # Replicas: 1,0 1,0 1,0
                    'MAP':[3,3], # Replicas: 1,1 1,1 1,1
                    'TP10':[1,5], # 0,3 1,0 0,2
                    'TP2':[2,7], # 1,2 0,3 1,2
                    'Leu9':[0,22]} # 0,7 0,8 0,7


    # Labels and values
    labels = list(behaviours.keys())
    translocation = [-v[0] for v in behaviours.values()]  # Negative for left bars
    insertion = [v[1] for v in behaviours.values()]       # Positive for right bars

    # Position for the labels
    y = np.arange(len(labels))

    # Calculate limits for better centering
    x_limit = max(max(abs(np.array(translocation))), max(insertion)) + 5  # Ensure equal padding

    # Plot
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.barh(y, translocation, color='orange', label='Translocation', edgecolor='black', align='center')
    ax.barh(y, insertion, color='orchid', label='Insertion', edgecolor='black', align='center')

    # Annotating the values
    for i, (t, ins) in enumerate(zip(translocation, insertion)):
        ax.text(t - 1, i, str(abs(t)), va='center', ha='right', color='k', fontsize=12)
        ax.text(ins + 1, i, str(ins), va='center', ha='left', color='k', fontsize=12)

    # Axis settings
    ax.set_yticks(y)
    ax.set_yticklabels(labels,fontsize=12)
    plt.xticks(fontsize=12)
    ax.axvline(0, color='black', linewidth=0.8)  # Add the vertical line at x=0
    ax.set_xlim(min(translocation) - 5, max(insertion) + 5)
    # ax.set_xlim(-x_limit, x_limit)               # Symmetric x-axis limits for centering
    ax.invert_yaxis()  # To have the last bar at the top like your image
    ax.set_xlabel("Number of events",fontsize=14)
    # ax.set_title("Translocation vs Insertion", fontsize=14)

    # Legend
    plt.legend(loc='upper right',fontsize=12)

    # Show
    plt.tight_layout()
    plt.savefig(f'{plots_folder}behaviours_8peptides.svg')
    plt.show()

In [97]:
def behavior_8peptides():
    """ This function plots the number of times each behavior occurs in the CompEL simulations with 8 peptides """
    behaviours_data = {
        'dq_0': {'Partitioning': 79, 'Insertion': 1, 'Translocation': 0},
        'dq_8': {'Partitioning': 67, 'Insertion': 13, 'Translocation': 0},
        'dq_12': {'Partitioning': 64, 'Insertion': 16, 'Translocation': 0},
        'dq_16': {'Partitioning': 74, 'Insertion': 37, 'Translocation': 9},
    }

    # Colors for the pie chart segments
    colors = ['#66b3ff', 'orchid', 'orange']

    # Function to format percentages and hide zeros
    def autopct_format(pct):
        return ('%1.0f%%' % pct) if pct > 0 else ''

    # Plotting pie charts for each dictionary
    fig, axes = plt.subplots(1, 5, figsize=(17, 18))

    for i, (key, values) in enumerate(behaviours_data.items()):
        totals = list(values.values())
        
        # Plot each pie chart without labels
        wedges, texts, autotexts = axes[i].pie(totals, autopct=autopct_format, startangle=90, colors=colors, wedgeprops = { 'linewidth' : 1, 'edgecolor' : 'k' })
        for autotext in autotexts: autotext.set_fontsize(14)
        # axes[i].set_title(f'DQ {key[3:]}',fontsize=16)
        # Add a white circle in the center to create a donut effect
        center_circle = plt.Circle((0, 0), 0.40, color='k', fc='k')
        axes[i].add_artist(center_circle)
        
        # Add the title inside the circle
        axes[i].text(0, 0, f'∆Q {key[3:]}', horizontalalignment='center', verticalalignment='center', fontsize=12, weight='bold',color='w')


    # Add a shared legend for all pie charts
    fig.legend(wedges, ['Partitioning', 'Insertion', 'Translocation'], loc='center right', bbox_to_anchor=(1.1, 0.5),fontsize=12)

    # Equal aspect ratio ensures that pies are drawn as circles
    plt.tight_layout()  # Adjust layout to fit the legend
    plt.savefig(f'{plots_folder}behaviours_8peptides.svg')
    plt.show()

## H Bonds

In [99]:
def analyse_hbond_gmx(pept_folder,repl_num):
    """ Calculate the hydrogen bonds with gromacs """
    if repl_num == 1:
        gmx_command_1 = f'(echo "Protein"; echo "Protein") | gmx hbond -f {pept_folder}step7_compel.xtc -s {pept_folder}step7_compel.tpr -num {pept_folder}hb_prot_prot_1.xvg -nthreads 30'
        gmx_command_2 = f'(echo "Protein"; echo "POPC") | gmx hbond -f {pept_folder}step7_compel.xtc -s {pept_folder}step7_compel.tpr -num {pept_folder}hb_prot_popc_1.xvg -nthreads 30'
        gmx_command_3 = f'(echo "Protein"; echo "TIP3") | gmx hbond -f {pept_folder}step7_compel.xtc -s {pept_folder}step7_compel.tpr -num {pept_folder}hb_prot_tip3_1.xvg -nthreads 30'
    else:
        gmx_command_1 = f'(echo "Protein"; echo "Protein") | gmx hbond -f {pept_folder}step7_compel_{repl_num}.xtc -s {pept_folder}step7_compel_{repl_num}.tpr -num {pept_folder}hb_prot_prot_{repl_num}.xvg -nthreads 30'
        gmx_command_2 = f'(echo "Protein"; echo "POPC") | gmx hbond -f {pept_folder}step7_compel_{repl_num}.xtc -s {pept_folder}step7_compel_{repl_num}.tpr -num {pept_folder}hb_prot_popc_{repl_num}.xvg -nthreads 30'
        gmx_command_3 = f'(echo "Protein"; echo "TIP3") | gmx hbond -f {pept_folder}step7_compel_{repl_num}.xtc -s {pept_folder}step7_compel_{repl_num}.tpr -num {pept_folder}hb_prot_tip3_{repl_num}.xvg -nthreads 30'
    subprocess.call(f'/bin/bash -c "source /usr/local/gromacs-2020.7/bin/GMXRC && {gmx_command_1}"', shell=True)
    subprocess.call(f'/bin/bash -c "source /usr/local/gromacs-2020.7/bin/GMXRC && {gmx_command_2}"', shell=True)
    subprocess.call(f'/bin/bash -c "source /usr/local/gromacs-2020.7/bin/GMXRC && {gmx_command_3}"', shell=True)

In [100]:
def create_hbond_csv(pept_folder,dq_num,pept_num,repl_num):
    """ Create a csv file with the hydrogen bonds data """
    df = pd.DataFrame()
    for file,abbr in [['prot_prot','PP'],['prot_popc','PL'],['prot_tip3','PW']]:
        with open(f'{pept_folder}hb_{file}_{repl_num}.xvg') as f:
            time_frames = []
            hbonds_num = []
            pairs_num = []
            for line in f:
                if line.startswith('#') or line.startswith('@'):
                    pass
                else:
                    parts = line.split()
                    time_frames.append(parts[0])
                    hbonds_num.append(parts[1])
                    pairs_num.append(parts[2])

            df[f'HB {abbr}'] = hbonds_num
            df[f'Pairs {abbr}'] = pairs_num
    df['Time'] = [int(i)/1000 for i in time_frames]
    df.to_csv(f'{data_folder}8.Hbonds/{dq_num}.{pept_num}.{repl_num}.csv')

In [103]:
def plot_hbonds(dq_num,pept_num,repl_num):
    """ Plot the H bonds formation for each replica """
    df = pd.read_csv(f'{data_folder}8.Hbonds/{dq_num}.{pept_num}.{repl_num}.csv',index_col=0).rolling(10).mean()
    plt.plot(df['Time'],df['HB PP'],label='Protein-Protein')
    plt.plot(df['Time'],df['HB PL'],label='Protein-Lipid')
    plt.plot(df['Time'],df['HB PW'],label='Protein-Water')
    plt.xlabel('Time (ns)')
    plt.ylabel('# H Bonds')
    plt.legend()
    plt.savefig(f'{plots_folder}8.Hbonds/{dq_num}.{pept_num}.{repl_num}.svg')
    plt.close()

In [101]:
def average_hbond(pept_num,dq_num=4):
    """ Calculate the average H bonds formation for all replicas """
    if dq_num == 4:
        df1 = pd.read_csv(f'{data_folder}8.Hbonds/{dq_num}.{pept_num}.1.csv',index_col=0)
        df2 = pd.read_csv(f'{data_folder}8.Hbonds/{dq_num}.{pept_num}.2.csv',index_col=0)
        df3 = pd.read_csv(f'{data_folder}8.Hbonds/{dq_num}.{pept_num}.3.csv',index_col=0)
    elif dq_num == 7:
        df1 = pd.read_csv(f'{data_folder}8.Hbonds/{dq_num}.{pept_num}.1_gmx.csv',index_col=0)
        df2 = pd.read_csv(f'{data_folder}8.Hbonds/{dq_num}.{pept_num}.2_gmx.csv',index_col=0)
        df3 = pd.read_csv(f'{data_folder}8.Hbonds/{dq_num}.{pept_num}.3_gmx.csv',index_col=0)
    df = pd.DataFrame()
    dfs = [df1,df2,df3]
    for num,dfi in enumerate(dfs,start=1):
        df[f'HB PP {num}'] = dfi['HB PP']
        df[f'HB PL {num}'] = dfi['HB PL']
        df[f'HB PW {num}'] = dfi['HB PW']
    df['HB PP Avg'] = df[['HB PP 1', 'HB PP 2', 'HB PP 3']].mean(axis=1)
    df['HB PL Avg'] = df[['HB PL 1', 'HB PL 2', 'HB PL 3']].mean(axis=1)
    df['HB PW Avg'] = df[['HB PW 1', 'HB PW 2', 'HB PW 3']].mean(axis=1)
    df['Time'] = df1['Time']
    df.to_csv(f'{data_folder}8.Hbonds/new_{dq_num}.{pept_num}_avg.csv')

In [104]:
def plot_average_hbonds(pept_num,dq_num=4):
    """ Plot the average H bonds formation for all replicas """
    df = pd.read_csv(f'{data_folder}8.Hbonds/{dq_num}.{pept_num}_avg.csv',index_col=0).rolling(10).mean()
    plt.plot(df['Time'],df['HB PP Avg'],label='Protein-Protein')
    plt.plot(df['Time'],df['HB PL Avg'],label='Protein-Lipid')
    plt.plot(df['Time'],df['HB PW Avg'],label='Protein-Water')
    plt.xlabel('Time (ns)')
    plt.ylabel('# H Bonds')
    plt.legend()
    plt.savefig(f'{plots_folder}8.Hbonds/{dq_num}.{pept_num}_avg.svg')
    plt.close()

In [107]:
def plot_all_hbonds(dq_num):
    """ Plot the average H bonds formation for all peptides in the same graph using mosaics"""
    # Create axis and load all dfs
    fig,axes = plt.subplots(figsize=(17,10))
    # This is used to assign each df to a variable called df1,df2,etc.
    for i in range(1, 6):
        if dq_num == 4:
            globals()[f'df{i}'] = pd.read_csv(f'{data_folder}8.Hbonds/{dq_num}.{i}_avg.csv', index_col=0)
        elif dq_num == 7:
            globals()[f'df{i}'] = pd.read_csv(f'{data_folder}8.Hbonds/new_{dq_num}.{i}_avg.csv', index_col=0)
    dfs = [df1,df2,df3,df4,df5] # Save into a list and use later
    columns_to_normalize = ['HB PP Avg', 'HB PL Avg', 'HB PW Avg']
    # Apply min-max normalization to each column
    for num,df in enumerate(dfs,start=1):
        df[columns_to_normalize] = df[columns_to_normalize].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
        df.to_csv(f'{data_folder}8.Hbonds/{dq_num}.{num}_avg_normalized.csv')
    # Now load the normalized df and save into list, just like before
    for i in range(1, 6):
        globals()[f'df{i}'] = pd.read_csv(f'{data_folder}8.Hbonds/{dq_num}.{i}_avg_normalized.csv', index_col=0).rolling(10).mean()
    dfs = [df1,df2,df3,df4,df5] # Save into a list and use later
    # Get the maximum value of all the dfs. The max value is the sum at every timestep of the 3 averages. We will divide by this value to calcualte the ratio.
    # max_value = int(max(max(df[['HB PP Avg','HB PL Avg','HB PW Avg']].sum(axis=1)) for df in dfs))

    # Set the global axis off
    axes.set_axis_off()
    # Create mosaic of the desired size and characteristics. F and G are only created to center E
    axes = fig.subplot_mosaic("""AABB
                            CCDD
                            FEEG""",sharex=True,sharey=True)
    keys = ['A','B','C','D','E']
    for num,key in enumerate(keys):
        # Add labels and set the y limits from 0 to 100
        # axes[key].set_xlabel('Time (ns)')
        # axes[key].set_ylabel('# H Bonds')
        # axes[key].set_ylim(0, 1550)
        # Use label_outer to remove redundant x and y labels
        axes[key].label_outer()
        # Add individual titles
        # axes[key].set_title(cpp_names[num])
    # I have created graphs F and G, but I don't want to use them. I just created them to be able to center plot E. So, now, we set the axis off for graphs F and G
    axes['F'].set_axis_off()
    axes['G'].set_axis_off()
    # Make sure that C and D keep their x-axis labels despite label_outer
    # axes['C'].set_xlabel('Time (ns)')
    # axes['D'].set_xlabel('Time (ns)')
    # axes['E'].set_xlabel('Time (ns)')
    # axes['E'].set_ylabel('# H Bonds')
    axes['C'].tick_params(labelbottom=True,labelsize=14)  # Ensure the tick labels are also visible
    axes['D'].tick_params(labelbottom=True,labelsize=14)
    axes['E'].tick_params(labelleft=True,labelsize=14)
    axes['A'].tick_params(labelleft=True,labelsize=14)
    # Loop over the dfs and the keys and plot
    for dfi,key in zip(dfs,keys):
        axes[key].plot(dfi['Time'],dfi[f'HB PP Avg'],color='purple')
        axes[key].plot(dfi['Time'],dfi[f'HB PL Avg'],color='orange')
        axes[key].plot(dfi['Time'],dfi[f'HB PW Avg'],color='cyan')
    # Create custom legend handles
    legend_handles = [Line2D([0], [0], color='purple', lw=2, label='Peptide'),
                    Line2D([0], [0], color='orange', lw=2, label='Peptide-Lipid'),
                    Line2D([0], [0], color='cyan', lw=2, label='Peptide-Water')]
    # Situate the legend where we want
    fig.legend(handles=legend_handles, loc='lower right', ncol=1, bbox_to_anchor=(0.95,0.15))
    plt.xticks(fontsize=30)
    plt.yticks(fontsize=30)
    plt.tight_layout()
    plt.savefig(f'{plots_folder}8.Hbonds/avg_{dq_num}.svg')
    plt.show()
    

## ALL

In [90]:
def analyse_everything(dq_num,start_pept,last_pept,start_repl=1,last_repl=2):
    """ This function is used to run all analysing/plotting functions at the same time """
    folder = f'{folders[dq_num]}/'
    pept_folders = sorted(glob.glob(f'{folder}/?.*'))

    for pept_num in range(start_pept,last_pept):
        for repl_num in range(start_repl,last_repl):
            pept_folder = f'{pept_folders[pept_num]}/'
            print(f'{pept_folder} Repl {repl_num}')
            # Define parameter files
            if dq_num < 7: # If DQ lower than 7 the name is:
                if pept_num == 0: parm_file = f'{pept_folder}system-kcl.gro' # If no peptide, then system-kcl.gro
                elif pept_num == 5: # If Leu9, then system-peptide.gro, since no Cl- added
                    if repl_num == 2 and dq_num == 0: # Special case
                        parm_file = f'{pept_folder}step6.6_equilibration_2.gro'
                    else:
                        parm_file = f'{pept_folder}system-peptide.gro'
                else: parm_file = f'{pept_folder}system-clpeptide.gro' # If peptide, then system-clpeptide.gro, since Cl- added
            else: # Different parameter files for DQ bigger than 6
                if pept_num == 5: # Leu9 has no Cl added, so different files
                    parm_file = f'{pept_folder}8peptides.gro'
                else: parm_file = f'{pept_folder}8peptides-cl.gro'
            #Define trajectory files
            if repl_num == 1:
                if pept_num == 0:
                    # if dq_num == 2 or dq_num == 4:
                    #     traj_file = [f'{pept_folder}step7_compel_first.xtc',f'{pept_folder}step7_compel_second.xtc']
                    # else:
                        traj_file = f'{pept_folder}step7_compel.xtc'
                elif pept_num == 5 and dq_num == 6: 
                    traj_file = [f'{pept_folder}step7_compel_first.xtc',f'{pept_folder}step7_compel_second.xtc']
                else: traj_file = f'{pept_folder}step7_compel.xtc'
            elif repl_num == 2: 
                if dq_num == 6:
                    # if pept_num == 0 or pept_num == 5:
                    #     traj_file = [f'{pept_folder}step7_compel_2_first.xtc',f'{pept_folder}step7_compel_2_second.xtc']
                    # else:
                        traj_file = f'{pept_folder}step7_compel_2.xtc'
                else:
                    traj_file = f'{pept_folder}step7_compel_2.xtc'
            else: 
                traj_file = f'{pept_folder}step7_compel_3.xtc'
                
            if repl_num == 1: center_traj = f'{pept_folder}step7_compel-whole-nojump-mol.xtc'
            else: center_traj = f'{pept_folder}step7_compel-whole-nojump-mol_{repl_num}.xtc'
            
            # Center and wrap trajectory
            # center_wrap_trajectory(pept_folder,repl_num)

            # Load variables needed to run analysis
            u = mda.Universe(parm_file,traj_file)
            c = mda.Universe(parm_file,center_traj)

            # Analysing function
            analyse_rmsd(c,dq_num,pept_num,repl_num)
            analyse_density(pept_folder,traj_file,dq_num,pept_num,repl_num)
            analyse_density_per_membrane(pept_folder,traj_file,dq_num,pept_num,repl_num)
            analyse_pore(c,pept_num,dq_num,interacting_membrane='upper',repl_num=repl_num)
            analyse_pore(c,pept_num,dq_num,interacting_membrane='lower',repl_num=repl_num)
            
            if pept_num != 0:
                # Secondary structure analysis
                ss_tcl_csv(dq_num,pept_num,repl_num)
                plot_ss_csv(u,dq_num,pept_num,repl_num)
                plot_avg_ss_csv(dq_num,pept_num)
                # Lipid analysis
                analyse_occupancy_lipid_part(u,parm_file=parm_file,pept_num=pept_num,dq_num=dq_num,repl_num=repl_num,traj_file=traj_file,lipid_part='head')
                analyse_occupancy_lipid_part(u,parm_file=parm_file,pept_num=pept_num,dq_num=dq_num,repl_num=repl_num,traj_file=traj_file,lipid_part='tail')
                for part in ['upper','lower']:
                    for lipid_part in ['head','tail']:
                        analyse_occupancy_lipid_part(u,traj_file,parm_file,part,dq_num,pept_num,repl_num)
                        plot_occupancy_lipid_part(dq_num,pept_num,repl_num,lipid_part,part)
                analyse_hbond_gmx(pept_folder,repl_num)
                create_hbond_csv(pept_folder,dq_num,pept_num,repl_num)
                plot_hbonds(dq_num,pept_num,repl_num)

            analyse_potentials(dq_num,pept_num,repl_num,pept_folder,traj_file)
            analyse_potentials_symm(dq_num,pept_num,repl_num,pept_folder,traj_file)

            ## Lipid analysis
            # analyse_lipid_order_parameters(u,dq_num,pept_num,repl_num)

            # Plotting functions
            plot_rmsd(dq_num,pept_num,repl_num)
            plot_density(dq_num,pept_num,repl_num)
            plot_density_per_membrane(dq_num,pept_num,repl_num)
            plot_pore(dq_num,pept_num,repl_num,sliding_window=10,interacting_membrane='upper')
            plot_pore(dq_num,pept_num,repl_num,sliding_window=10,interacting_membrane='lower')

            plot_potentials_symm(dq_num,pept_num,repl_num)

            plot_lipid_order_parameters(dq_num,pept_num,repl_num)
        # Run the analysis/plots for all replicas (avg)
        # plot_avg_ss_csv(dq_num,pept_num)
        average_occupancies(dq_num,pept_num,part)
        for part in ['head','tail']:
            average_occupancies_lipid_part(dq_num,pept_num,part,lipid_part)
            plot_avg_occupancy_lipid_part(dq_num,pept_num,repl_num,lipid_part,part)
        average_hbond(pept_num,dq_num)
        plot_average_hbonds(pept_num,dq_num)